In [1]:
from typing import Dict
import torch
from torch.utils.data import DataLoader
from .datasets import build_datasets

def _worker_init_fn(worker_id):
    # rend les workers déterministes
    worker_seed = torch.initial_seed() % 2**32
    import random, numpy as np
    random.seed(worker_seed)
    np.random.seed(worker_seed)

def build_dataloaders(cfg: Dict):
    name = cfg["name"]
    root = cfg.get("root", "./data/cache")
    img_size = int(cfg.get("img_size", 32))
    augment = bool(cfg.get("augment", True))
    seed = int(cfg.get("seed", 42))
    val_size = int(cfg.get("val_size", 5000))

    train_ds, val_ds, test_ds, num_classes = build_datasets(
        name=name, root=root, img_size=img_size, augment=augment, seed=seed, val_size=val_size
    )

    batch_size = int(cfg.get("batch_size", 128))
    num_workers = int(cfg.get("num_workers", 4))
    pin_memory = torch.cuda.is_available()
    persistent_workers = num_workers > 0

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True, drop_last=True,
        num_workers=num_workers, pin_memory=pin_memory,
        worker_init_fn=_worker_init_fn, persistent_workers=persistent_workers
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory,
        worker_init_fn=_worker_init_fn, persistent_workers=persistent_workers
    )
    test_loader = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory,
        worker_init_fn=_worker_init_fn, persistent_workers=persistent_workers
    )

    meta = {
        "num_classes": num_classes,
        "image_size": img_size,
        "channels": 3,  # on force 3 canaux pour unifier MNIST et CIFAR-10
        "train_len": len(train_ds),
        "val_len": len(val_ds),
        "test_len": len(test_loader.dataset),
    }
    return train_loader, val_loader, test_loader, meta



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/edwardlucyszyn/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/edwardlucyszyn/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/edwardlucyszyn/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

ImportError: attempted relative import with no known parent package